# Resources
* [Overview of diff](https://blog.jcoglan.com/2017/02/12/the-myers-diff-algorithm-part-1/)
* [Patience diff intro](https://blog.jcoglan.com/2017/09/19/the-patience-diff-algorithm/#:~:text=What%20it%20really%20is%20is,like%20Myers%20on%20those%20pieces.&text=Patience%20diff%20splits%20this%20problem,exactly%20once%20in%20both%20versions.)

In [16]:
with open('a.py','r') as f:
    a_file = f.read()

with open('b.py','r') as f:
    b_file = f.read()


a_file = a_file.split("\n")
b_file = b_file.split("\n")

class Slice:
    def __init__(self, a_low, a_high, b_low, b_high, a_data, b_data):
        self.a_low = a_low
        self.b_low = b_low
        self.a_data = a_data
        self.b_data = b_data
        

# initialize the slice, a is the whole content of file one and b is whole content of file two
start_slice = Slice(0, len(a_file), 0, len(b_file), a_file, b_file)

def find_unique_match(slice):
    # line content: [number of appearances in A, number of appearances in B, position in A, position in B]
    line_counts = {}
    matches = []
    a = slice.a_data
    b = slice.b_data
    
    for i in range(len(a)):
        if a[i] not in line_counts:
            position = slice.a_low + i
            line_counts[a[i]] = [1,0,position,None]
        else:
            line_counts[a[i]][0] += 1
    
    for i in range(len(b)):
        # b[i] is not in line_counts indicates there's no matches
        if b[i] in line_counts:
            position = slice.b_low + i
            line_counts[b[i]][1] += 1
            line_counts[b[i]][3] = position
    
    for line in line_counts:
        if line_counts[line][:2] == [1,1]:
            matches.append((line,line_counts[line][2:]))
    
    return matches

matches = find_unique_match(start_slice)
for match in matches:
    print(match)


('def save_to_csv(output_file,results):', [0, 0])
('    Save result to a csv file', [2, 8])
('    Args:', [4, 10])
('        output_file: string, output filename', [5, 11])
('        results: list, result of all the matches', [6, 12])
("    with open(output_file,'w') as file:", [9, 1])
('        csv_file = csv.writer(file)', [10, 2])
('        for result in results:', [11, 3])
('            for match in result[1]:', [12, 4])
("                file.write(result[0] + ',')", [13, 5])
('                csv_file.writerow(match[:-1])', [14, 6])
('def main():', [16, 17])
("    cwd = os.getcwd() + '/'", [17, 18])
('    results = [] # stores the result of all the matches for each file', [18, 19])
('    # parsing input argument', [20, 21])
("    parser = argparse.ArgumentParser(description='Extract dates from txt files')", [21, 22])
("    parser.add_argument('-i', '--input', type=str, help='input file directory path')", [22, 23])
("    parser.add_argument('-o', '--output',type=str, default='outp